In [14]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from lstm_sine import SineData

<IPython.core.display.Javascript object>

In [5]:
%load_ext autoreload
%load_ext nb_black
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [19]:
if torch.cuda.is_available():
    device = "cuda:1"
else:
    print("Gpu not found")
    device = "cpu"

<IPython.core.display.Javascript object>

In [51]:
x = torch.arange(100, device=device)
y = np.sin(x.cpu()).to(device)
x = torch.zeros(100, 1)

<IPython.core.display.Javascript object>

In [43]:
class LSTMModel(nn.Module):
    def __init__(self, nodes=10, layers=1, device=device):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(1, nodes, layers)

    def forward(self, x):
        h_n = torch.zeros(x.size(0), 1, device=device)

<IPython.core.display.Javascript object>

In [44]:
model = LSTMModel().to(device)

<IPython.core.display.Javascript object>